In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Overview

Choose whatever language you're most comfortable with to solve these problems.

# Exercise

The ACME inc. tool supply company manages its operations with 3 csv files:

1. `customers.csv` keeps customer information:
    * `id` is a numeric customer id
    * `firstname` is the customer's first name
    * `lastname` is the customer's last name
2. `products.csv` keeps product info:
    * `id` is a numeric product id
    * `name` is the human-readable name
    * `cost` is the product cost in euros
3. `orders.csv` keeps order information:
    * `id` is a numeric order id
    * `customer` is the numeric id of the customer who created the order
    * `products` is a space-separated list of product ids ordered by the customer

Manually dealing with those files is hard and error-prone, and they've asked for your help writing some code to make their lives easier.



In [2]:
#loading and cleaning data
customers = pd.read_csv('interview/customers.csv')
customers.rename(columns={'id':'customer_id'}, inplace=True)
products = pd.read_csv('interview/products.csv')
products.rename(columns={'id':'products_id'}, inplace=True)
orders = pd.read_csv('interview/orders.csv')
orders.rename(columns={'id':'order_id', 'customer':'customer_id', 'products':'products_id'}, inplace=True)

orders_new = orders.values.tolist()

### Task 1

Right now the `orders.csv` doesn't have total order cost information.

We need to use the data in these files to emit a `order_prices.csv` file with the following columns:
* `id` the numeric id of the order
* `euros` the total cost of the order


In [3]:
#we're going to make an example to identify the process
input_example = pd.DataFrame({
    'order_id':[0,0,0,0,1],
    'products_id':['1 0','0','0','1','1'],
    'cost':[3,5,3,5,5]
}).values.tolist()

In [4]:
intermediate_example = input_example = pd.DataFrame({
    'order_id':[0,0,0,0,1],
    'cost':[3,5,3,5,5]
})

In [5]:
output_example = pd.DataFrame({
    'order_id':[0,1],
    'cost':[16,5]
})

In [6]:
from pandas._testing import assert_frame_equal
output = intermediate_example.groupby(by='order_id').sum().reset_index()
assert_frame_equal(output,output_example)

In [7]:
#denormalizing products from orders
def splitting_by_space(list_of_spaces, index_order):
    result = pd.DataFrame(list_of_spaces.str.split(' ').tolist(),index=index_order).stack().reset_index()
    return result.drop('level_1', axis=1).rename(columns={0: 'products_id'})


In [8]:
orders_denormalize = splitting_by_space(orders['products_id'], orders['order_id'])

In [9]:
orders_denormalize['products_id'] = orders_denormalize['products_id'].astype('int')

In [10]:
#merging to obtain cost per product
orders_with_cost = pd.merge(orders_denormalize, products[['products_id', 'cost']], on='products_id', how='left')
orders_with_cost.head()

,order_id,products_id,cost
0,0,1,6.490396
1,0,0,2.981164
2,0,1,6.490396
3,0,0,2.981164
4,1,0,2.981164


In [11]:
def get_orders_prices(orders_data):
    intermediate = orders_data.drop('products_id', axis=1)
    output = intermediate.groupby('order_id').sum().reset_index()
    output.rename(columns={'cost':'euros'})
    return output

In [12]:
order_prices = get_orders_prices(orders_with_cost)

In [13]:
#order_prices.to_csv('order_prices.csv', index=False)

,order_id,cost
0,0,18.943120
1,1,61.425421
2,2,23.145479
3,3,34.399455
4,4,45.540896
5,5,15.356456
6,6,73.094931
7,7,25.221718
8,8,18.865689
9,9,37.190652
